In [7]:
#----------------------------------------------------------------------
# Purpose:  Condition an Airline dataset by filtering out NAs where the
#           departure delay in the input dataset is unknown.
#
#           Then treat anything longer than minutesOfDelayWeTolerate
#           as delayed.
#----------------------------------------------------------------------

In [8]:
import sys
sys.path.insert(1, '/Users/rpeck/Source/h2o-3/h2o-py')
import h2o

In [9]:
h2o.init(ip="127.0.0.1", port=54321, strict_version_check=False)

H2O cluster uptime:,20 minutes 53 seconds 171 milliseconds
H2O cluster version:,3.7.0.3300
H2O cluster name:,rpeck
H2O cluster total nodes:,1
H2O cluster total memory:,3.56 GB
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [10]:
air = h2o.import_frame(h2o.locate("/Users/rpeck/Source/h2o-3/smalldata/airlines/allyears2k_headers.zip"))


Parse Progress: [##################################################] 100%


In [11]:
numRows, numCols = air.dim
print "Original dataset rows: {0}, columns: {1}".format(numRows, numCols)

x_cols = ["Month", "DayofMonth", "DayOfWeek", "CRSDepTime", "CRSArrTime", "UniqueCarrier", "CRSElapsedTime", "Origin", "Dest", "Distance"]
y_col = "SynthDepDelayed"

noDepDelayedNAs = air[air["DepDelay"].isna() == 0]
rows, cols = noDepDelayedNAs.dim
print "New dataset rows: {0}, columns: {1}".format(rows, cols)

Original dataset rows: 43978, columns: 31


EnvironmentError: h2o-py got an unexpected HTTP status code:
 400 Bad Request (method = POST; url = http://127.0.0.1:54321/99/Rapids). 
detailed error messages: Temp ID py_1 already exists

In [ ]:
minutesOfDelayWeTolerate = 15
noDepDelayedNAs.cbind(noDepDelayedNAs["DepDelay"] > minutesOfDelayWeTolerate)
noDepDelayedNAs[numCols] = noDepDelayedNAs[numCols-1].asfactor()
noDepDelayedNAs.setName(numCols,y_col)

In [ ]:
gbm = h2o.gbm(x=noDepDelayedNAs[x_cols], y=noDepDelayedNAs[y_col], distribution="bernoulli")
gbm.show()